In [1]:
#!/usr/bin/env python3
import json
import numpy as np
import requests
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
###Create Tools for code
def append_file_to_list(file):
    list = []
    f = open(file, 'r')
    for line in f:
        line = line.strip('\n')
        line = line.split('\t')
        list.append(line)
    return list

In [3]:
#Pull in Data from JSON
CIViC = requests.get('https://civic.genome.wustl.edu/api/variants?count=1000000').json()['records']
DoCM = requests.get('http://docm.genome.wustl.edu/api/v1/variants?count=1000000').json()
variants_capture = requests.get('https://civic.genome.wustl.edu/api/panels/captureseq/qualifying_variants?minimum_score=0').json()['records']

In [61]:
#open error file for deep learning
error_data = append_file_to_list('deep_learning/errors_dl.tsv')
error_pd = pd.read_csv('deep_learning/errors_dl.tsv', sep='\t')

In [5]:
#Open SOID files
eligible_SOIDs = []
f = open('SOID_terms.txt', 'r')
for line in f:
    line = line.strip('\n')
    line = line.strip()
    eligible_SOIDs.append(line)

In [62]:
print('Number of Discrepencies: ', len(error_pd[(error_pd.error=='False Negative') | (error_pd.error=='False Positive')]))
print('Number of False Positives: ', len(error_pd[(error_pd.error=='False Positive')]))
print('Number of False Negatives: ', len(error_pd[(error_pd.error=='False Negative')]))

Number of Discrepencies:  4378
Number of False Positives:  1659
Number of False Negatives:  2719


In [63]:
false_neg = []
false_pos = []
true_pos = []
for item in error_data:
    if item[2] == 'False Negative':
        false_neg.append([item[7], item[8], item[9], item[10], item[11], item[12], item[5], item[0], item[1]])
    if item[2] == 'False Positive':
        false_pos.append([item[7], item[8], item[9], item[10], item[11], item[12], item[5], item[0], item[1]])
    if item[2] == 'True Positive':
        true_pos.append([item[7], item[8], item[9], item[10], item[11], item[12], item[5], item[0], item[1]])
            

In [46]:
#pull docm coordinates for overlap
DoCM_coordinates = []
for item in DoCM:
    variant = item['variant']
    start = item['start']
    stop = item['stop']
    chrom = item['chr']
    reference = item['read']
    diseases = item['diseases']
    name = item['gene']
    citation = len(item['pubmed_sources'])
    DoCM_coordinates.append([chrom, start, stop, reference, variant, name, citation])

DoCM_clinical_variants_FN = []
for item in false_neg:
    for exon in DoCM_coordinates:
        if str(item[0]) == str(exon[0]) and int(item[1]) == int(exon[1]) and int(item[2]) == int(exon[2]) and str(item[3]) == str(exon[3]) and str(item[4]) == str(exon[4]):
            DoCM_clinical_variants_FN.append([item[0], item[1], item[2], item[3], item[4], exon[5], item[5], item[6], item[7], item[8], exon[6]])

DoCM_clinical_variants_FP = []            
for item in false_pos:
    for exon in DoCM_coordinates:
        if str(item[0]) == str(exon[0]) and int(item[1]) == int(exon[1]) and int(item[2]) == int(exon[2]) and str(item[3]) == str(exon[3]) and str(item[4]) == str(exon[4]):
            DoCM_clinical_variants_FP.append([item[0], item[1], item[2], item[3], item[4], exon[5], item[5], item[6], item[7], item[8], exon[6]])
            
DoCM_clinical_variants_TP = []            
for item in true_pos:
    for exon in DoCM_coordinates:
        if str(item[0]) == str(exon[0]) and int(item[1]) == int(exon[1]) and int(item[2]) == int(exon[2]) and str(item[3]) == str(exon[3]) and str(item[4]) == str(exon[4]):
            DoCM_clinical_variants_TP.append([item[0], item[1], item[2], item[3], item[4], exon[5], item[5], item[6], item[7], item[8], exon[6]])
            
print('Number of DoCM Annotations:', len(DoCM))
print(len(DoCM_clinical_variants_FN))
print(len(DoCM_clinical_variants_FP))
print(len(DoCM_clinical_variants_TP))


Number of DoCM Annotations: 1364
10
14
226


In [53]:
#Print DoCM Variants to list
final_FN = {}
for item in DoCM_clinical_variants_FN:
    code = str(item[0] + item[1] + item[2] + item[3] + item[4])
    if code in final_FN.keys():
        final_FN[code][10] += item[10] 
    if code not in final_FN.keys():
        final_FN[code] = ''
        final_FN[code] = item
print('False Negative Overlap with DoCM Annotations:', len(final_FN))

#Print DoCM Variants to list
final_FP = {}
for item in DoCM_clinical_variants_FP:
    code = str(item[0] + item[1] + item[2] + item[3] + item[4])
    if code in final_FP.keys():
        final_FP[code][10] += item[10] 
    if code not in final_FP.keys():
        final_FP[code] = ''
        final_FP[code] = item
print('False Positive Overlap with DoCM Annotations:', len(final_FP))

#Print DoCM Variants to list
final_TP = {}
for item in DoCM_clinical_variants_TP:
    code = str(item[0] + item[1] + item[2] + item[3] + item[4])
    if code in final_TP.keys():
        final_TP[code][10] += item[10] 
    if code not in final_TP.keys():
        final_TP[code] = ''
        final_TP[code] = item
print('True Positive Overlap with DoCM Annotations:', len(final_TP))
pubs_TP = 0
for k,v in final_TP.items():
    pubs_TP += int(v[10])
print('Publications associated with True Positives is: ',pubs_TP)


clinical = open('deep_learning/DoCM_variants_DL_FN.txt', 'w')
clinical.write('DoCM Annotation Overlap with False Negatives' + '\n')
clinical.write('Chr' + '\t' + 'Start' + '\t'+ 'Stop' + '\t' + 'Ref' + '\t' + 'Var' + '\t' + 'Gene' + '\t' + 'Disease' + '\t' + 'Probability' + '\t' + 'MR Call' + '\t' + 'Classifier' + '\t' + 'Publications'+ '\n')
for k,v in final_FN.items():
    for bugger in v:
        clinical.write(str(bugger) + '\t')
    clinical.write('\n')
clinical.close()

clinical = open('deep_learning/DoCM_variants_DL_FP.txt', 'w')
clinical.write('DoCM Annotation Overlap with False Positives' + '\n')
clinical.write('Chr' + '\t' + 'Start' + '\t'+ 'Stop' + '\t' + 'Ref' + '\t' + 'Var' + '\t' + 'Gene' + '\t' + 'Disease' + '\t' + 'Probability' + '\t' + 'MR Call' + '\t' + 'Classifier' + '\t' + 'Publications'+ '\n')
for k,v in final_FP.items():
    for bugger in v:
        clinical.write(str(bugger) + '\t')
    clinical.write('\n')
clinical.close()

False Negative Overlap with DoCM Annotations: 8
False Positive Overlap with DoCM Annotations: 12
True Positive Overlap with DoCM Annotations: 94
Publications associated with True Positives is:  9846


In [60]:
#Pull items from CIViC
score = {'A':5, 'B':4, 'C':3, 'D':2, 'E':1}

capture_sequence_probes = [] #create empty list for capture sequence probes
for k in range(0, len(variants_capture)): #iterate through API and pull all eligible variants
    counts = [0,0,0,0,0]
    gene = variants_capture[k]['entrez_name']  #Call Gene name
    variant = variants_capture[k]['name'] #call variant
    soid = variants_capture[k]['variant_types'][0]['so_id'] #call soid
    variant_type = variants_capture[k]['variant_types'][0]['name'] #call variant type
    evidence = variants_capture[k]['evidence_items'] #pull evidence items
    evidence_statements = len(variants_capture[k]['evidence_items']) #pull number of evidence statements
    chrom = variants_capture[k]['coordinates']['chromosome'] #call chrom
    start = variants_capture[k]['coordinates']['start'] #call start
    stop = variants_capture[k]['coordinates']['stop'] #call stop
    ref = variants_capture[k]['coordinates']['reference_bases']
    var = variants_capture[k]['coordinates']['variant_bases']
    evidence_type = [] #set list for evidence types
    evidence_scores = [] #set list for evidence scores
    top_evidences = [] #set list for top evidence level
    for item in evidence: #iterate through the evidence items
        if item['evidence_type'] not in evidence_type: #see if the evidence type is already there
            evidence_type.append(item['evidence_type']) #if it is not append it
        if item['evidence_type'] == 'Predictive':
            if item['clinical_significance'] == 'Sensitive' or item['clinical_significance'] == 'Sensitivity':
                counts[0] += 1
            elif item['clinical_significance'] == 'Resistance or Non-Response': # or 'Adverse' in item['clinical_significance']:
                counts[1] += 1
            elif item['clinical_significance'] == 'Adverse Response': # or 'Adverse' in item['clinical_significance']:
                counts[1] += 1
        elif item['evidence_type'] == 'Prognostic':
            counts[2] += 1
        elif item['evidence_type'] == 'Diagnostic':
            counts[3] += 1
        elif item['evidence_type'] == 'Predisposing':
            counts[4] += 1
        trust_rating = int(item['rating'] or 0) #make the trust rating either what is listed or 0
        evidence_level = int(score[item['evidence_level']]) #make the evidence level the value from the score dictionary
        evidence_scores.append(evidence_level * trust_rating) #calculate the Evidence Score
        if item['evidence_level'] != '[]': #find the evidence levels that are not blank
            top_evidences.append(item['evidence_level'].strip()) #add to the list
    #pull the maximum evidence level
    if 'A' in top_evidences:
        top_evidence = 'A'
    elif 'B' in top_evidences:
        top_evidence = 'B'
    elif 'C' in top_evidences:
        top_evidence = 'C'
    elif 'D' in top_evidences:
        top_evidence = 'D'
    else:
        top_evidence = 'E'
    evidence_score = sum(evidence_scores) #sum the evidence scores to get a CIVic Score
    evidence_types = ', '.join(evidence_type) #format the evidence types
    if soid in eligible_SOIDs:
        capture_sequence_probes.append([chrom, start,stop, ref, var, gene, soid, variant_type, variant, top_evidence, evidence_types, evidence_statements, evidence_score, counts])

print('Number of Eligible Loci within CIViC:', len(capture_sequence_probes))

sen = 0
res = 0
pro = 0
dia = 0
pre = 0

for item in capture_sequence_probes:
    sen += int(item[13][0])
    res += int(item[13][1])
    pro += int(item[13][2])
    dia += int(item[13][3])
    pre += int(item[13][4])
        
print('True Positive Sensitivity = ', sen) 
print('True Positive Resistance = ', res)   
print('True Positive Prognosis = ', pro)   
print('True Positive Diagnostic = ', dia)   
print('True Positive Predisposition = ', pre)   

Number of Eligible Loci within CIViC: 425
True Positive Sensitivity =  772
True Positive Resistance =  547
True Positive Prognosis =  176
True Positive Diagnostic =  50
True Positive Predisposition =  307


In [41]:
#Elucidate False Negatives that have exact and direct overlap with CIViC

capture_overlap_unique_exact_FN = []
capture_overlap_unique_about_FN = []

for item in false_neg:
    for exon in capture_sequence_probes:
        code = [item[0], item[1], item[2], item[3], item[4], exon[5], item[5], item[6], item[7], item[8], exon[8], exon[9], exon[10], exon[11], exon[12], exon[13]]
        if str(item[0]) == str(exon[0]) and int(item[1]) == int(exon[1])and int(item[2]) == int(exon[2]) and str(item[3]) == str(exon[3]) and str(item[4]) == str(exon[4]) and code not in capture_overlap_unique_exact_FN:
            capture_overlap_unique_exact_FN.append(code)
        if str(item[0]) == str(exon[0]) and int(item[1]) >= int(exon[1]) and int(item[2]) <= int(exon[2]) and code not in capture_overlap_unique_about_FN:
            capture_overlap_unique_about_FN.append(code)

print('Number of Exact Unique Overlaps with CIViC:', len(capture_overlap_unique_exact_FN))
print('Number of Total Unique Overlaps with CIViC:', len(capture_overlap_unique_about_FN))

final_FN = {}

for item in capture_overlap_unique_about_FN:
    code = str(item[0] + item[1] + item[2] + item[3] + item[4] + item[7])
    if code in final_FN.keys():
        variant = final_FN[code][10].split(',')
        type_thing =  final_FN[code][12].split(',')
        if item[10] not in variant:
            final_FN[code][10] += ', '
            final_FN[code][10] += item[10]
        if item[12] not in type_thing:
            final_FN[code][12] += ', '
            final_FN[code][12] += item[12]
        final_FN[code][13] += item[13]
        final_FN[code][14] += item[14]
    if code not in final_FN.keys():
        final_FN[code] = ''
        final_FN[code] = item

print('Overlap with CIViC Variants:', len(final_FN))

clinical = open('deep_learning/clinical_variants_False_Negatives.txt', 'w')
clinical.write('False Negative Overlap with CIViC' + '\n')
clinical.write("Chr" + '\t' + "Start" + '\t' + "Stop" + '\t' + "Ref" + '\t' 
               + "Var" + '\t' + 'Gene' + '\t' + "Disease" + '\t' + "Confidence" 
               + '\t' + "MR Call" + '\t' + "Classifier Call" + '\t' + "Variant" 
               + '\t' + "Top Evidence" + '\t' + "Evidence Type" + '\t' 
               + "Evidence Items" + '\t' + "CIViC Score" + '\t' + "Sensitivity"
               + '\t' + "Resistance" + '\t' + "Prognosis" + '\t' + "Diagnosis"
               + '\t' + "Predisposing" + "\n")
for k,v in final_FN.items():
    for item in range(15):
        clinical.write(str(v[item]) + '\t')
    for item in v[15]:
        clinical.write(str(item) + '\t')
    clinical.write('\n')
clinical.close()


Number of Exact Unique Overlaps with CIViC: 3
Number of Total Unique Overlaps with CIViC: 51
Overlap with CIViC Variants: 40


In [40]:
#Elucidate False Negatives that have exact and direct overlap with CIViC

capture_overlap_unique_exact_FP = []
capture_overlap_unique_about_FP = []

for item in false_pos:
    for exon in capture_sequence_probes:
        code = [item[0], item[1], item[2], item[3], item[4], exon[5], item[5], item[6], item[7], item[8], exon[8], exon[9], exon[10], exon[11], exon[12], exon[13]]
        if str(item[0]) == str(exon[0]) and int(item[1]) == int(exon[1]) and int(item[2]) == int(exon[2]) and str(item[3]) == str(exon[3]) and str(item[4]) == str(exon[4]) and code not in capture_overlap_unique_exact_FP:
            capture_overlap_unique_exact_FP.append(code)
        if str(item[0]) == str(exon[0]) and int(item[1]) >= int(exon[1]) and int(item[2]) <= int(exon[2]) and code not in capture_overlap_unique_about_FP:
            capture_overlap_unique_about_FP.append(code)

print('Number of Exact Unique Overlaps with CIViC for False Positives:', len(capture_overlap_unique_exact_FP))
print('Number of Total Unique Overlaps with CIViC for False Positives:', len(capture_overlap_unique_about_FP))

final_FP = {}

for item in capture_overlap_unique_about_FP:
    code = str(item[0] + item[1] + item[2] + item[3] + item[4] + item[7])
    if code in final_FP.keys():
        variant = final_FP[code][10].split(',')
        type_thing =  final_FP[code][12].split(',')
        if item[10] not in variant:
            final_FP[code][10] += ', '
            final_FP[code][10] += item[10]
        if item[12] not in type_thing:
            final_FP[code][12] += ', '
            final_FP[code][12] += item[12]
        final_FP[code][13] += item[13]
        final_FP[code][14] += item[14]
    if code not in final_FP.keys():
        final_FP[code] = ''
        final_FP[code] = item

print('False Positive Overlap with CIViC Variants:', len(final_FP))

clinical = open('deep_learning/clinical_variants_False_Positive.txt', 'w')
clinical.write('False Positive Overlap with CIViC' + '\n')
clinical.write("Chr" + '\t' + "Start" + '\t' + "Stop" + '\t' + "Ref" + '\t' 
               + "Var" + '\t' + 'Gene' + '\t' + "Disease" + '\t' + "Confidence" 
               + '\t' + "MR Call" + '\t' + "Classifier Call" + '\t' + "Variant" 
               + '\t' + "Top Evidence" + '\t' + "Evidence Type" + '\t' 
               + "Evidence Items" + '\t' + "CIViC Score" + '\t' + "Sensitivity"
               + '\t' + "Resistance" + '\t' + "Prognosis" + '\t' + "Diagnosis"
               + '\t' + "Predisposing" + "\n")
for k,v in final_FP.items():
    for item in range(15):
        clinical.write(str(v[item]) + '\t')
    for item in v[15]:
        clinical.write(str(item) + '\t')
    clinical.write('\n')
clinical.close()


Number of Exact Unique Overlaps with CIViC for False Positives: 5
Number of Total Unique Overlaps with CIViC for False Positives: 65
False Positive Overlap with CIViC Variants: 53


In [42]:
#Determine Total Number of Variants that overlap with CIViC for all calls
error_data_bed = {}
for idx,item in enumerate(error_data):
    error_data_bed[idx] = [item, [0,0,0,0,0]]
    for exon in capture_sequence_probes:
        if str(item[7]) == str(exon[0]) and int(item[8]) >= int(exon[1]) and int(item[9]) <= int(exon[2]):
            error_data_bed[idx][1][0] += exon[13][0]
            error_data_bed[idx][1][1] += exon[13][1]
            error_data_bed[idx][1][2] += exon[13][2]
            error_data_bed[idx][1][3] += exon[13][3]
            error_data_bed[idx][1][4] += exon[13][4]

In [ ]:
both = 0
manual_review_somatic = 0
deep_learning_somatic = 0

both_list = {}
manual_review_somatic_list = {}
deep_learning_somatic_list = {}

variants = {}
for item in range(len(error_data) - 1):
    if item not in variants.keys():
        variants[item] = error_data[item + 1]

for k,v in error_data_bed.items():
    if v[0][0] == 's' and v[0][1] == 's':
        both += 1
        both_list[k] = v
    elif v[0][0] == 's' and v[0][1] != 's':
        manual_review_somatic += 1
        manual_review_somatic_list[k] = v
    elif v[0][0] != 's' and v[0][1] == 's':
        deep_learning_somatic += 1
        deep_learning_somatic_list[k] = v

print('Variants in Both: ', both)
print('Variants in Manual Review Only: ', manual_review_somatic)
print('Variants in Deep Learning Only: ', deep_learning_somatic)
print()

mr_counts = 0
deep_counts = 0
both_counts = 0

mr_clinical = [0,0,0,0,0]
dl_clinical = [0,0,0,0,0]
both_clinical = [0,0,0,0,0]

mr_clinical_list = {}
dl_clinical_list = {}
both_clinical_list = {}

for k,v in both_list.items():
    if v[1] != [0,0,0,0,0]:
        both_counts += 1
        both_clinical_list[k] = v
        both_clinical[0] += v[1][0]
        both_clinical[1] += v[1][1]
        both_clinical[2] += v[1][2]
        both_clinical[3] += v[1][3]
        both_clinical[4] += v[1][4]

for k,v in manual_review_somatic_list.items():
    if v[1] != [0,0,0,0,0]:
        mr_counts += 1
        mr_clinical_list[k] = v
        mr_clinical[0] += v[1][0]
        mr_clinical[1] += v[1][1]
        mr_clinical[2] += v[1][2]
        mr_clinical[3] += v[1][3]
        mr_clinical[4] += v[1][4]

for k,v in deep_learning_somatic_list.items():
    if v[1] != [0,0,0,0,0]:
        deep_counts += 1
        dl_clinical_list[k] = v
        dl_clinical[0] += v[1][0]
        dl_clinical[1] += v[1][1]
        dl_clinical[2] += v[1][2]
        dl_clinical[3] += v[1][3]
        dl_clinical[4] += v[1][4]
        
print('Clinical Variants in Both: ', both_counts)
print('Clinical Variants in Manual Review Only: ', mr_counts)
print('Clinical Variants in Deep Learning Only: ', deep_counts)
print()
print('Clinical Variants in Both: ', both_clinical)
print('Clinical Variants in Manual Review Only: ', mr_clinical)
print('Clinical Variants in Deep Learning Only: ', dl_clinical)

avg = []
for k,v in mr_clinical_list.items():
    avg.append(float(v[0][4]))
    
file = open('manual_review_to_review.tsv', 'w')
for k,v in mr_clinical_list.items():
    file.write(str(v[0][0]) + '\t' + str(v[0][6]) + '\t' + str(v[0][7]) + '\t' + str(v[0][8]) + '\t' +  str(v[0][9]) + '\n')
file.close()